---

# 0. Imports

---

In [28]:
# Basic imports
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

# Functional imports
import json
#import pyarrow
from pickle import dump
import gc

# Sklearn basic imports
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# XGBoost imports
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score

pd.set_option('display.max_columns', None)

---

# 1. Data Collection

---

In [81]:
#df = pd.read_parquet("../data/raw/Combined_Flights_2018.parquet")
#df_2019 = pd.read_parquet("../data/raw/Combined_Flights_2019.parquet")
#df_2020 = pd.read_parquet("../data/raw/Combined_Flights_2020.parquet")
#df = pd.read_parquet("../data/raw/Combined_Flights_2021.parquet")
#df_2022 = pd.read_parquet("../data/raw/Combined_Flights_2022.parquet")
df = pd.read_parquet("../data/raw/Combined_Flights_2022.parquet")

In [127]:
#data_frames = [df_2018, df_2019, df_2020, df_2021, df_2022]
#data_frames = [df_2019, df_2020, df_2021, df_2022]
#data_frames = [df_2020, df_2021, df_2022]
#data_frames = [df_2021, df_2022]
#df = pd.concat(data_frames, ignore_index=True)

In [128]:
#del df_2018, df_2019, df_2020, df_2021, df_2022, data_frames
#del df_2019, df_2020, df_2021, df_2022, data_frames
#del df_2020, df_2021, df_2022, data_frames
#del df_2021, df_2022, data_frames
#gc.collect()

28

In [82]:
df["Month"].unique()

array([4, 2, 7, 5, 6, 1, 3], dtype=int64)

In [83]:
df.head(3)

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,1228.0,0.0,40.0,72.0,65.0,212.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4301,C5,20445,C5,N21144,4301,11921,1192102,31921,"Grand Junction, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,-1.0,1100-1159,17.0,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,848.0,0.0,55.0,77.0,80.0,295.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4299,C5,20445,C5,N16170,4299,12206,1220605,32206,"Harlingen/San Benito, TX",TX,48,Texas,74,12266,1226603,31453,"Houston, TX",TX,48,Texas,74,0.0,-1.0,0700-0759,16.0,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,1636.0,0.0,47.0,70.0,82.0,251.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4298,C5,20445,C5,N21144,4298,11413,1141307,30285,"Durango, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,-1.0,1500-1559,21.0,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0


---

# 2. Exploration and Data cleaning

---

In [31]:
print('Our dataframe contains {} rows and it has {} features.'.format(len(df), df.shape[1]))

Our dataframe contains 4078318 rows and it has 61 features.


In [32]:
print(df.columns)

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

## 2.1 Clasificación de los features

### a. Información redundante o irrelevante para el objetivo
Estas columnas no aportan valor predictivo o contienen información que no estará disponible antes del despegue.

| **Columna(s)**                                                                                                         | **Descripción**                                                                                              |
|------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------|
| **'DepTime', 'DepDelayMinutes', 'DepDelay'**                                                                             | Dependen del retraso real al momento de la salida, que es lo que queremos predecir.                         |
| **'ArrTime', 'ArrDelayMinutes', 'AirTime', 'ActualElapsedTime', 'ArrDelay', 'ArrDel15', 'ArrivalDelayGroups'**           | Información posterior al despegue.                                                                          |
| **'Cancelled', 'Diverted', 'DivAirportLandings'**                                                                        | Indican si el vuelo fue cancelado o desviado, lo cual no es útil para predecir retrasos antes del despegue.  |
| **'WheelsOff', 'WheelsOn', 'TaxiOut', 'TaxiIn'**                                                                         | Información posterior al despegue.                                                                          |
| **'Tail_Number'**                                                                                                        | No aporta información relevante y puede introducir ruido.                                                   |
| **'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID'**| IDs redundantes que pueden ser reemplazados con información más significativa como el nombre del aeropuerto o la ciudad. |
| **'OriginWac', 'DestWac'**                                                                                               | Códigos geográficos redundantes.                                                                            |



### b. Información que podría ser útil pero requiere validación
Estas columnas pueden tener correlación con retrasos, pero debemos evaluar su utilidad.

| **Columna(s)**                                                                                                         | **Descripción**                                                                                              |
|------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------|
| **'FlightDate'**                                                                                                         | Podría ser relevante si extraemos características como feriados, temporada alta, etc.                       |
| **'Flight_Number_Marketing_Airline', 'Flight_Number_Operating_Airline'**                                                 | Dato granular que podría ser redundante con la aerolínea y menos práctico para usuarios en una página web.   |
| **'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners'**                                               | Podrían correlacionarse con retrasos, pero hay que evitar redundancia con otras columnas relacionadas a la aerolínea. |
| **'DOT_ID_Marketing_Airline', 'DOT_ID_Operating_Airline', 'IATA_Code_Marketing_Airline', 'IATA_Code_Operating_Airline'** | IDs redundantes con el nombre de la aerolínea.                                                              |
| **'DistanceGroup'**                                                                                                      | Es categórica, pero podría ser redundante con **'Distance'**.                                                 |
| **'DepartureDelayGroups'**                                                                                               | Representa rangos de retrasos, pero depende del retraso real, por lo que no será útil.                      |



### c. Información clave para el modelo
Estas columnas probablemente tendrán mayor impacto en la predicción.

| **Columna(s)**                                                                 | **Descripción**                                                                                         |
|--------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------|
| **'Airline', 'Operating_Airline'**                                               | Las aerolíneas pueden tener diferentes tasas de puntualidad.                                           |
| **'Origin', 'Dest', 'OriginCityName', 'DestCityName', 'OriginState', 'DestState'**| Lugares de origen y destino son cruciales, ya que algunos aeropuertos tienen mayores tasas de retraso. |
| **'CRSDepTime', 'CRSArrTime', 'DepTimeBlk', 'ArrTimeBlk'**                       | Hora programada y bloque horario pueden correlacionarse con la congestión del aeropuerto y los retrasos.|
| **'Distance'**                                                                   | La distancia entre el origen y el destino puede afectar la probabilidad de retrasos.                   |
| **'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek'**                        | Información temporal que puede capturar tendencias estacionales o semanales.                           |



## 2.2 Propuesta de columnas finales
Variables seleccionadas para el modelo.

| **Columna(s)**             | **Descripción**                               |
|----------------------------|-----------------------------------------------|
| **'Airline' (o 'Operating_Airline')** | Aerolínea del vuelo.                         |
| **'Origin'**                 | Aeropuerto de origen.                        |
| **'Dest'**                   | Aeropuerto de destino.                       |
| **'OriginCityName'**         | Ciudad de origen.                            |
| **'DestCityName'**           | Ciudad de destino.                           |
| **'OriginState'**            | Estado de origen.                            |
| **'DestState'**              | Estado de destino.                           |
| **'CRSDepTime'**             | Hora programada de salida.                   |
| **'DepTimeBlk'**             | Bloque horario de salida.                    |
| **'Distance'**               | Distancia entre origen y destino.            |
| **'Year'**                   | Año del vuelo.                               |
| **'Quarter'**                | Trimestre del vuelo.                         |
| **'Month'**                  | Mes del vuelo.                               |
| **'DayofMonth'**             | Día del mes del vuelo.                       |
| **'DayOfWeek'**              | Día de la semana del vuelo.                  |


## 2.3 Eliminating Irrelevant Features and dealing with null values

Eliminaremos algunos features antes de continuar con nuestro EDA para ahorrar en recursos y tiempo

In [33]:
# 1 = Weekday, 0 = Weekend
Day_Week_Type = []
for value in df['DayOfWeek']:
    if value in (0,1,2,3,4):
        Day_Week_Type.append(1) # Weekday
    else:
        Day_Week_Type.append(0) # Weekend
df['WeekType'] = Day_Week_Type
df.head(3)

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings,WeekType
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,1228.0,0.0,40.0,72.0,65.0,212.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4301,C5,20445,C5,N21144,4301,11921,1192102,31921,"Grand Junction, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,-1.0,1100-1159,17.0,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0,1
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,848.0,0.0,55.0,77.0,80.0,295.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4299,C5,20445,C5,N16170,4299,12206,1220605,32206,"Harlingen/San Benito, TX",TX,48,Texas,74,12266,1226603,31453,"Houston, TX",TX,48,Texas,74,0.0,-1.0,0700-0759,16.0,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0,1
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,1636.0,0.0,47.0,70.0,82.0,251.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4298,C5,20445,C5,N21144,4298,11413,1141307,30285,"Durango, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,-1.0,1500-1559,21.0,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0,1


In [34]:
columnas_finales = [
    # Features
    'Airline', 'Origin', 'Dest', 'OriginCityName', 'DestCityName',
    'OriginStateName', 'DestStateName', 'CRSDepTime', 'DepTimeBlk', 'Distance',
    'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'WeekType',
    
    # Targets
    'DepDel15', 'DepDelayMinutes'
]


In [35]:
df = df[columnas_finales]
df.head(3)

,Airline,Origin,Dest,OriginCityName,DestCityName,OriginStateName,DestStateName,CRSDepTime,DepTimeBlk,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,WeekType,DepDel15,DepDelayMinutes
0,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,"Grand Junction, CO","Denver, CO",Colorado,Colorado,1133,1100-1159,212.0,2022,2,4,4,1,1,0.0,0.0
1,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,"Harlingen/San Benito, TX","Houston, TX",Texas,Texas,732,0700-0759,295.0,2022,2,4,4,1,1,0.0,0.0
2,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,"Durango, CO","Denver, CO",Colorado,Colorado,1529,1500-1559,251.0,2022,2,4,4,1,1,0.0,0.0


In [36]:
df.shape

(4078318, 18)

In [37]:
df.dtypes

Airline             object
Origin              object
Dest                object
OriginCityName      object
DestCityName        object
OriginStateName     object
DestStateName       object
CRSDepTime           int64
DepTimeBlk          object
Distance           float64
Year                 int64
Quarter              int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
WeekType             int64
DepDel15           float64
DepDelayMinutes    float64
dtype: object

## 2.3 Analisis de nuestra dataframe

In [38]:
df.describe()

,CRSDepTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,WeekType,DepDel15,DepDelayMinutes
count,4.078318e+06,4.078318e+06,4078318.0,4.078318e+06,4.078318e+06,4.078318e+06,4.078318e+06,4.078318e+06,3.957823e+06,3.957823e+06
mean,1.329587e+03,7.978657e+02,2022.0,1.741207e+00,4.083803e+00,1.571131e+01,4.005566e+00,5.708395e-01,2.184625e-01,1.601494e+01
std,4.904801e+02,5.914742e+02,0.0,7.037563e-01,1.998918e+00,8.760122e+00,2.007050e+00,4.949564e-01,4.132030e-01,5.231498e+01
min,1.000000e+00,3.100000e+01,2022.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.140000e+02,3.680000e+02,2022.0,1.000000e+00,2.000000e+00,8.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.320000e+03,6.430000e+02,2022.0,2.000000e+00,4.000000e+00,1.600000e+01,4.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,1.735000e+03,1.035000e+03,2022.0,2.000000e+00,6.000000e+00,2.300000e+01,6.000000e+00,1.000000e+00,0.000000e+00,1.100000e+01
max,2.359000e+03,5.095000e+03,2022.0,3.000000e+00,7.000000e+00,3.100000e+01,7.000000e+00,1.000000e+00,1.000000e+00,7.223000e+03


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4078318 entries, 0 to 590541
Data columns (total 18 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Airline          object 
 1   Origin           object 
 2   Dest             object 
 3   OriginCityName   object 
 4   DestCityName     object 
 5   OriginStateName  object 
 6   DestStateName    object 
 7   CRSDepTime       int64  
 8   DepTimeBlk       object 
 9   Distance         float64
 10  Year             int64  
 11  Quarter          int64  
 12  Month            int64  
 13  DayofMonth       int64  
 14  DayOfWeek        int64  
 15  WeekType         int64  
 16  DepDel15         float64
 17  DepDelayMinutes  float64
dtypes: float64(3), int64(7), object(8)
memory usage: 591.2+ MB


***Coclusion:***

Nuestro data frame contiene los siguientes tipos de datos:

- ****Float****     --> 3
- ****Integer****   --> 6
- ****String****    --> 8



## 2.4 Buscando duplicados, valores vacios y lidiando con ellos

In [40]:
#Encontrando valores duplicados
print(f"Valores duplicados: {df.duplicated().sum()}")

#Eliminando valores duplicados
df = df.drop_duplicates()
print("\nValores duplicados eliminados!")

Valores duplicados: 768

Valores duplicados eliminados!


In [41]:
#Encontrando valores NaN
print(f"Valores Nan encontrados: \nColumnas          Cantidad\n{df.isna().sum()[df.isna().sum()>0]}")

df = df.dropna()
print("\nValores NaN eliminados!")

Valores Nan encontrados: 
Columnas          Cantidad
DepDel15           120483
DepDelayMinutes    120483
dtype: int64

Valores NaN eliminados!


***Conclusion:***

De aproximadamente 4M de filas se encontrarons solo 573 filas duplicadas y 762978 de valores nulos

Sabiendo estos numeros lo que haremos seremos eliminarlos para que no afecte en el analisis de datos. Esta decision fue tomada considerando que al ser un porcentaje bajo de data no afectara al momento del analisis

---

# 3. Dividing our dataset in umercial and categorical, Encoding and dividing our dataframe for the clasiffication and regression model

## 3.1 Categorical and numerical

In [42]:
# Understanding how many type of features we have
print(f"Estos son los tipos de datos que tenemos en nuestro data set: {df.dtypes.unique()}")

Estos son los tipos de datos que tenemos en nuestro data set: [dtype('O') dtype('int64') dtype('float64')]


In [43]:
num = ["float64", "int64"]
cat = ['O']

df_num = df.select_dtypes(num)
df_cat = df.select_dtypes(cat)

## 3.2 Encoding our data set

In [44]:
# Creating encoders for categorical features and saving them as JSON files. All files prefixed with 'enc'
# contain the encoding dictionaries for each categorical feature.
df_enc = df.copy()

for column in df_cat.columns:
    unique_values = list(df_cat[column].unique())
    globals()[f"{column}_enc"] = dict(zip(unique_values, range(len(unique_values))))

    json.dump(globals()[f"{column}_enc"], open(f'../data/interim/enc_{column}.json', 'w'))


In [45]:
# Replacing the values in our categorical features to our encoded values (numerical)
for column in df_cat.columns:
    df_enc[column] = df_enc[column].map(json.load(open(f'../data/interim/enc_{column}.json')))

In [46]:
df_enc.head(3)

,Airline,Origin,Dest,OriginCityName,DestCityName,OriginStateName,DestStateName,CRSDepTime,DepTimeBlk,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,WeekType,DepDel15,DepDelayMinutes
0,0,0,0,0,0,0,0,1133,0,212.0,2022,2,4,4,1,1,0.0,0.0
1,0,1,1,1,1,1,1,732,1,295.0,2022,2,4,4,1,1,0.0,0.0
2,0,2,0,2,0,0,0,1529,2,251.0,2022,2,4,4,1,1,0.0,0.0


In [47]:
del df, df_cat, df_num

gc.collect()

64

## 3.3 Escalar los datos de distancia y CRSDepTime, para que no valore los datos debido a su reango de diferencia entre ellos

In [48]:
scaler = StandardScaler()
df_enc[['Distance', 'CRSDepTime']] = scaler.fit_transform(df_enc[['Distance', 'CRSDepTime']])

In [49]:
df_enc.head()

,Airline,Origin,Dest,OriginCityName,DestCityName,OriginStateName,DestStateName,CRSDepTime,DepTimeBlk,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,WeekType,DepDel15,DepDelayMinutes
0,0,0,0,0,0,0,0,-0.398844,0,-0.989899,2022,2,4,4,1,1,0.0,0.0
1,0,1,1,1,1,1,1,-1.217656,1,-0.850014,2022,2,4,4,1,1,0.0,0.0
2,0,2,0,2,0,0,0,0.409758,2,-0.924170,2022,2,4,4,1,1,0.0,0.0
3,0,3,2,3,2,1,2,0.217817,3,-0.713499,2022,2,4,4,1,1,0.0,0.0
4,0,2,0,2,0,0,0,-0.394760,0,-0.924170,2022,2,4,4,1,1,0.0,0.0


## 3.4 Dividing our data sets para el modelo de clasificacion y el de regresion

In [50]:
#Dataset para modelo de clasificacion
df_clasificacion = df_enc.copy()

#Dataset para modelo de regresion
df_regresion = df_enc[df_enc['DepDel15'] == 1].copy()


print("\nDistribución de la variable objetivo para clasificación (DepDel15):\n")
print(df_clasificacion['DepDel15'].value_counts(normalize=True))

print("\nEstadísticas de la variable objetivo para regresión (DepDelayMinutes):\n")
print(df_regresion['DepDelayMinutes'].describe())


Distribución de la variable objetivo para clasificación (DepDel15):

DepDel15
0.0    0.781496
1.0    0.218504
Name: proportion, dtype: float64

Estadísticas de la variable objetivo para regresión (DepDelayMinutes):

count    864636.000000
mean         68.341686
std          94.804174
min          15.000000
25%          24.000000
50%          41.000000
75%          77.000000
max        7223.000000
Name: DepDelayMinutes, dtype: float64


## 3.5 Revisar datos procesados


In [51]:
del df_enc

gc.collect()

0

In [52]:
print("Datos procesados para clasificación:")
df_clasificacion.head()

Datos procesados para clasificación:


,Airline,Origin,Dest,OriginCityName,DestCityName,OriginStateName,DestStateName,CRSDepTime,DepTimeBlk,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,WeekType,DepDel15,DepDelayMinutes
0,0,0,0,0,0,0,0,-0.398844,0,-0.989899,2022,2,4,4,1,1,0.0,0.0
1,0,1,1,1,1,1,1,-1.217656,1,-0.850014,2022,2,4,4,1,1,0.0,0.0
2,0,2,0,2,0,0,0,0.409758,2,-0.924170,2022,2,4,4,1,1,0.0,0.0
3,0,3,2,3,2,1,2,0.217817,3,-0.713499,2022,2,4,4,1,1,0.0,0.0
4,0,2,0,2,0,0,0,-0.394760,0,-0.924170,2022,2,4,4,1,1,0.0,0.0


In [53]:
print("\nDatos procesados para regresión:")
df_regresion.head()


Datos procesados para regresión:


,Airline,Origin,Dest,OriginCityName,DestCityName,OriginStateName,DestStateName,CRSDepTime,DepTimeBlk,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,WeekType,DepDel15,DepDelayMinutes
18,0,4,12,4,12,0,4,0.822226,9,-0.012387,2022,2,4,4,1,1,1.0,29.0
21,0,4,10,4,10,0,0,-0.792935,4,-0.924170,2022,2,4,4,1,1,1.0,15.0
29,0,4,20,4,20,0,11,0.434261,2,-0.634287,2022,2,4,4,1,1,1.0,25.0
32,0,4,19,4,19,0,10,0.003415,8,-0.273619,2022,2,4,4,1,1,1.0,319.0
37,0,9,23,9,23,5,12,0.850813,9,-0.861812,2022,2,4,4,1,1,1.0,79.0


---

# 4. Hacer el split de train and test

In [54]:
# Variables predictoras y objetivo para clasificación
X_clasificacion = df_clasificacion.drop(columns=['DepDel15', 'DepDelayMinutes'])
y_clasificacion = df_clasificacion['DepDel15']

# Variables predictoras y objetivo para regresión
X_regresion = df_regresion.drop(columns=['DepDel15', 'DepDelayMinutes'])
y_regresion = df_regresion['DepDelayMinutes']

# Dividir datos para clasificación (80% entrenamiento, 20% prueba)
X_train_clas, X_test_clas, y_train_clas, y_test_clas = train_test_split(
    X_clasificacion, y_clasificacion, test_size=0.2, random_state=42, stratify=y_clasificacion
)

# Dividir datos para regresión (80% entrenamiento, 20% prueba)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regresion, y_regresion, test_size=0.2, random_state=42
)

# Revisar tamaños de los conjuntos
print("Clasificación:")
print(f"  Tamaño entrenamiento: {X_train_clas.shape}, Tamaño prueba: {X_test_clas.shape}")
print("\nRegresión:")
print(f"  Tamaño entrenamiento: {X_train_reg.shape}, Tamaño prueba: {X_test_reg.shape}")


Clasificación:
  Tamaño entrenamiento: (3165653, 16), Tamaño prueba: (791414, 16)

Regresión:
  Tamaño entrenamiento: (691708, 16), Tamaño prueba: (172928, 16)


# 5. Machine Learning

## 5.1 Modelo de clasificacion

# MODELO SIN HIPERPARAMETRIZACION

In [ ]:
'''# 1. Crear el modelo de XGBoost
xgb_clasificador = XGBClassifier(
    objective='binary:logistic',  # Clasificación binaria
    eval_metric='logloss',       # Métrica para evaluar la pérdida
    use_label_encoder=False,     # Deshabilitar codificación automática
    random_state=42
)

# 2. Entrenar el modelo con los datos de entrenamiento
xgb_clasificador.fit(X_train_clas, y_train_clas)

# 3. Predecir en el conjunto de prueba
y_pred_clas = xgb_clasificador.predict(X_test_clas)

# 4. Evaluar el modelo
print("Reporte de clasificación:")
print(classification_report(y_test_clas, y_pred_clas))

print("\nMatriz de confusión:")
print(confusion_matrix(y_test_clas, y_pred_clas))

print("\nExactitud del modelo:")
print(f"{accuracy_score(y_test_clas, y_pred_clas) * 100:.2f}%")
'''

c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:38:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.81      0.97      0.88    618487
         1.0       0.64      0.17      0.27    172927

    accuracy                           0.80    791414
   macro avg       0.72      0.57      0.57    791414
weighted avg       0.77      0.80      0.75    791414


Matriz de confusión:
[[602023  16464]
 [143902  29025]]

Exactitud del modelo:
79.74%


# MODELO CON SMOTE

In [ ]:
'''from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score

# 1. Aplicar oversampling con SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# 2. Crear el modelo con ajuste de ponderación
xgb_clasificador_bal = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    scale_pos_weight=len(y_train_clas[y_train_clas == 0]) / len(y_train_clas[y_train_clas == 1]),  # Ponderación
    random_state=42
)

# 3. Entrenar el modelo con los datos balanceados
xgb_clasificador_bal.fit(X_train_clas_bal, y_train_clas_bal)

# 4. Predecir en el conjunto de prueba
y_pred_clas_bal = xgb_clasificador_bal.predict(X_test_clas)
y_pred_proba_clas_bal = xgb_clasificador_bal.predict_proba(X_test_clas)[:, 1]

# 5. Evaluar el modelo
print("Reporte de clasificación (con balanceo):")
print(classification_report(y_test_clas, y_pred_clas_bal))

print("\nMatriz de confusión (con balanceo):")
print(confusion_matrix(y_test_clas, y_pred_clas_bal))

print("\nExactitud del modelo (con balanceo):")
print(f"{accuracy_score(y_test_clas, y_pred_clas_bal) * 100:.2f}%")

print("\nROC-AUC del modelo:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_clas_bal):.4f}")
'''

c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:44:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Reporte de clasificación (con balanceo):
              precision    recall  f1-score   support

         0.0       0.90      0.38      0.53    618487
         1.0       0.28      0.85      0.42    172927

    accuracy                           0.48    791414
   macro avg       0.59      0.61      0.47    791414
weighted avg       0.76      0.48      0.51    791414


Matriz de confusión (con balanceo):
[[232933 385554]
 [ 25481 147446]]

Exactitud del modelo (con balanceo):
48.06%

ROC-AUC del modelo:
0.7101


# XGBOOST CLASSIFIER CON GRIDSEARCHCV

In [55]:
# Crear el modelo de XGBoost
xgb_clasificador = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,
)

# Entrenar el modelo con los datos de entrenamiento
xgb_clasificador.fit(X_train_clas, y_train_clas)

# Predecir en el conjunto de prueba
y_pred_clas = xgb_clasificador.predict(X_test_clas)
y_pred_proba_clas = xgb_clasificador.predict_proba(X_test_clas)[:, 1]

# Calcular resultados
conf_matrix = confusion_matrix(y_test_clas, y_pred_clas)
classification_rep = classification_report(y_test_clas, y_pred_clas, output_dict=True)
accuracy = accuracy_score(y_test_clas, y_pred_clas)

# Calcular la curva ROC y el valor AUC
fpr, tpr, thresholds = roc_curve(y_test_clas, y_pred_proba_clas)
roc_auc = auc(fpr, tpr)

# Crear el DataFrame para el reporte de clasificación y redondear a 2 decimales
classification_df = pd.DataFrame(classification_rep).transpose().round(2)

# Crear el gráfico
fig, ax = plt.subplots(2, 2, figsize=(16, 16))

# Subplot 1: Matriz de confusión como mapa de calor
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", ax=ax[0, 0])
ax[0, 0].set_title("Matriz de Confusión", fontsize=14)
ax[0, 0].set_xlabel("Predicción", fontsize=12)
ax[0, 0].set_ylabel("Valor Real", fontsize=12)

# Subplot 2: Reporte de clasificación en estilo tabla
ax[0, 1].axis("tight")
ax[0, 1].axis("off")

# Dibujar el texto de exactitud
exact_text = f"Exactitud del Modelo: {accuracy * 100:.2f}%"
ax[0, 1].text(
    0.5, 1.2, exact_text, fontsize=16, color="blue", ha="center", transform=ax[0, 1].transAxes
)

# Dibujar la tabla con el reporte de clasificación
table = ax[0, 1].table(
    cellText=classification_df.values,
    colLabels=classification_df.columns,
    rowLabels=classification_df.index,
    loc="center",
    cellLoc="center",
    colLoc="center",
)
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)  # Escalar la tabla para mejor visualización

# Subplot 3: Curva ROC
ax[1, 0].plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {roc_auc:.2f})")
ax[1, 0].plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
ax[1, 0].set_xlim([0.0, 1.0])
ax[1, 0].set_ylim([0.0, 1.05])
ax[1, 0].set_title("Curva ROC", fontsize=14)
ax[1, 0].set_xlabel("Tasa de Falsos Positivos (FPR)", fontsize=12)
ax[1, 0].set_ylabel("Tasa de Verdaderos Positivos (TPR)", fontsize=12)
ax[1, 0].legend(loc="lower right", fontsize=12)

# Subplot 4 vacío (puedes usarlo para otros gráficos en el futuro)
ax[1, 1].axis("off")

# Ajustar diseño general
plt.tight_layout()
plt.show()

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'scale_pos_weight': 3.5765676028503317}

Reporte de clasificación (modelo optimizado):
              precision    recall  f1-score   support

         0.0       0.88      0.70      0.78    618487
         1.0       0.38      0.64      0.48    172927

    accuracy                           0.69    791414
   macro avg       0.63      0.67      0.63    791414
weighted avg       0.77      0.69      0.71    791414


Matriz de confusión (modelo optimizado):
[[435898 182589]
 [ 61833 111094]]

ROC-AUC del modelo optimizado:
0.7354

Exactitud del modelo (umbral ajustado):
69.12%


# XGBOOST REGRESSOR CON GRIDSEARCHCV

In [57]:
'''# Parámetros para búsqueda de hiperparámetros
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Modelo base
xgb_regressor = XGBRegressor(
    objective='reg:squarederror',
    random_state=42
)

# Búsqueda de hiperparámetros con GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_regressor,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Métrica de evaluación para regresión
    cv=3,  # Validación cruzada de 3 pliegues
    verbose=1,
    n_jobs=-1
)

# Entrenar el modelo
grid_search.fit(X_train_reg, y_train_reg)

# Mostrar los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

# Usar el mejor modelo para predecir
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_reg)

# Evaluación del modelo
mse = mean_squared_error(y_test_reg, y_pred)
r2 = r2_score(y_test_reg, y_pred)

print("\nEvaluación del modelo optimizado:")
print(f"MSE (Error cuadrático medio): {mse:.2f}")
print(f"R^2 (Coeficiente de determinación): {r2:.2f}")'''

Fitting 3 folds for each of 48 candidates, totalling 144 fits
Mejores parámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 1.0}

Evaluación del modelo optimizado:
MSE (Error cuadrático medio): 8798.54
R^2 (Coeficiente de determinación): 0.05


# XGBOOST REGRESSOR CON GRIDSEARCHCV SMOTE

In [62]:
'''num_samples = X_train_reg.shape[0]
max_neighbors = min(2, num_samples - 1)  # n_neighbors <= n_samples - 1

smote = SMOTE(random_state=42, k_neighbors=max_neighbors)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_reg, y_train_reg)
# Parámetros para búsqueda de hiperparámetros
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Modelo base
xgb_regressor = XGBRegressor(
    objective='reg:squarederror',
    random_state=42
)

# Búsqueda de hiperparámetros con GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_regressor,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Entrenar el modelo con los datos resampleados
grid_search.fit(X_train_resampled, y_train_resampled)

# Mostrar los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

# Usar el mejor modelo para predecir
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_reg)

# Evaluación del modelo
mse = mean_squared_error(y_test_reg, y_pred)
r2 = r2_score(y_test_reg, y_pred)

print("\nEvaluación del modelo optimizado:")
print(f"MSE (Error cuadrático medio): {mse:.2f}")
print(f"R^2 (Coeficiente de determinación): {r2:.2f}")'''

ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 3, n_samples_fit = 1, n_samples = 1

# RANDOM UNDERS AMPLER

In [67]:
'''from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Submuestreo de los datos de entrenamiento
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_clas, y_train_clas)

# Parámetros para buscar el mejor ajuste
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'scale_pos_weight': [1, len(y_train_resampled[y_train_resampled == 0]) / len(y_train_resampled[y_train_resampled == 1])]
}

# Crear el modelo base
xgb_clasificador_tuned = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(
    estimator=xgb_clasificador_tuned,
    param_grid=param_grid,
    scoring='roc_auc',  # Usamos ROC-AUC como métrica
    cv=3,  # Validación cruzada de 3 pliegues
    verbose=1,
    n_jobs=-1
)

# Entrenar la búsqueda con datos submuestreados
grid_search.fit(X_train_resampled, y_train_resampled)

# Mostrar los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

# Usar el mejor modelo para predecir
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_clas)
y_pred_proba_tuned = best_model.predict_proba(X_test_clas)[:, 1]

# Evaluación
print("\nReporte de clasificación (modelo optimizado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (modelo optimizado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo optimizado:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_tuned):.4f}")

print("\nExactitud del modelo (umbral ajustado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")
'''

Fitting 3 folds for each of 24 candidates, totalling 72 fits


c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:28:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'scale_pos_weight': 1}

Reporte de clasificación (modelo optimizado):
              precision    recall  f1-score   support

         0.0       0.88      0.70      0.78    618487
         1.0       0.38      0.64      0.48    172927

    accuracy                           0.69    791414
   macro avg       0.63      0.67      0.63    791414
weighted avg       0.77      0.69      0.71    791414


Matriz de confusión (modelo optimizado):
[[434867 183620]
 [ 61468 111459]]

ROC-AUC del modelo optimizado:
0.7357

Exactitud del modelo (umbral ajustado):
69.03%


# RANDOM OVER SAMPLER

In [68]:
'''from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Oversampling con RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_clas, y_train_clas)

# Parámetros para buscar el mejor ajuste
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'scale_pos_weight': [1]  # No es necesario ajustar este parámetro con datos balanceados
}

# Crear el modelo base
xgb_clasificador_tuned = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(
    estimator=xgb_clasificador_tuned,
    param_grid=param_grid,
    scoring='roc_auc',  # Usamos ROC-AUC como métrica
    cv=3,  # Validación cruzada de 3 pliegues
    verbose=1,
    n_jobs=-1
)

# Entrenar la búsqueda con datos oversampled
grid_search.fit(X_train_resampled, y_train_resampled)

# Mostrar los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

# Usar el mejor modelo para predecir
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_clas)
y_pred_proba_tuned = best_model.predict_proba(X_test_clas)[:, 1]

# Evaluación
print("\nReporte de clasificación (modelo optimizado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (modelo optimizado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo optimizado:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_tuned):.4f}")

print("\nExactitud del modelo (umbral ajustado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")
'''

Fitting 3 folds for each of 12 candidates, totalling 36 fits


c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:36:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'scale_pos_weight': 1}

Reporte de clasificación (modelo optimizado):
              precision    recall  f1-score   support

         0.0       0.88      0.71      0.78    618487
         1.0       0.38      0.64      0.48    172927

    accuracy                           0.69    791414
   macro avg       0.63      0.67      0.63    791414
weighted avg       0.77      0.69      0.72    791414


Matriz de confusión (modelo optimizado):
[[436914 181573]
 [ 62040 110887]]

ROC-AUC del modelo optimizado:
0.7358

Exactitud del modelo (umbral ajustado):
69.22%


# MUCHOS HIPERPARAMETROS

In [65]:
'''from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import numpy as np

# Parámetros ampliados para búsqueda
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'scale_pos_weight': [1, 2, 5, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5]
}

# Crear el modelo base
xgb_clasificador_tuned = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Búsqueda de hiperparámetros con F1-score
grid_search = GridSearchCV(
    estimator=xgb_clasificador_tuned,
    param_grid=param_grid,
    scoring='f1',  # Cambiar métrica de optimización a F1
    cv=3,  # Validación cruzada de 3 pliegues
    verbose=1,
    n_jobs=-1
)

# Entrenar la búsqueda
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X_train_clas, y_train_clas)

# Mostrar los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

# Usar el mejor modelo para predecir
y_pred_tuned = grid_search.best_estimator_.predict(X_test_clas)
y_pred_proba_tuned = grid_search.best_estimator_.predict_proba(X_test_clas)[:, 1]

# Evaluar el modelo
print("\nReporte de clasificación (modelo optimizado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (modelo optimizado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nExactitud del modelo (optimizado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")

# Ajuste del umbral de decisión
print("\nAjuste de umbrales para maximizar F1-score:")
y_pred_proba = grid_search.best_estimator_.predict_proba(X_test_clas)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.1)  # Probar diferentes umbrales
for threshold in thresholds:
    y_pred_adjusted = (y_pred_proba >= threshold).astype(int)
    f1 = f1_score(y_test_clas, y_pred_adjusted)
    print(f"Threshold: {threshold:.1f}, F1-score: {f1:.4f}")
'''

Iniciando búsqueda de hiperparámetros...
Fitting 3 folds for each of 5184 candidates, totalling 15552 fits


KeyboardInterrupt: 

# GRIDSEARCH CON XGBOOST SIN SMOTE 2022 y 2021

In [30]:
'''# Parámetros para buscar el mejor ajuste
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'scale_pos_weight': [1, len(y_train_clas[y_train_clas == 0]) / len(y_train_clas[y_train_clas == 1])]
}

# Crear el modelo base
xgb_clasificador_tuned = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(
    estimator=xgb_clasificador_tuned,
    param_grid=param_grid,
    scoring='roc_auc',  # Usamos ROC-AUC como métrica
    cv=3,  # Validación cruzada de 3 pliegues
    verbose=1,
    n_jobs=-1
)

# Entrenar la búsqueda
grid_search.fit(X_train_clas, y_train_clas)

# Mostrar los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

# Usar el mejor modelo para predecir
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_clas)
y_pred_proba_tuned = best_model.predict_proba(X_test_clas)[:, 1]

# Evaluación
print("\nReporte de clasificación (modelo optimizado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (modelo optimizado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo optimizado:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_tuned):.4f}")

print("\nExactitud del modelo (umbral ajustado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")
'''

Fitting 3 folds for each of 24 candidates, totalling 72 fits


c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:54:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'scale_pos_weight': 1}

Reporte de clasificación (modelo optimizado):
              precision    recall  f1-score   support

         0.0       0.83      0.98      0.90   1644104
         1.0       0.65      0.14      0.23    387781

    accuracy                           0.82   2031885
   macro avg       0.74      0.56      0.56   2031885
weighted avg       0.79      0.82      0.77   2031885


Matriz de confusión (modelo optimizado):
[[1615223   28881]
 [ 334932   52849]]

ROC-AUC del modelo optimizado:
0.7382

Exactitud del modelo (umbral ajustado):
82.09%


# EL MISMO QUE EL ANTERIOR PERO CON 2022 Y 2021 Y USANDO SOLO LOS HIPER PARAMETROS. SIN EL GRID SEARCH

In [138]:
'''from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

# 2. Configurar el clasificador XGBoost
xgb_clasificador = XGBClassifier(
    objective='binary:logistic',  # Clasificación binaria
    eval_metric='logloss',       # Métrica para evaluar la pérdida
    use_label_encoder=False,     # Deshabilitar codificación automática
    random_state=42,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=200,
    scale_pos_weight=3.5765676028503317
)

# 3. Entrenar el modelo con los datos balanceados
xgb_clasificador.fit(X_train_clas, y_train_clas)

# 4. Obtener las probabilidades de predicción en el conjunto de prueba
y_pred_proba = xgb_clasificador.predict_proba(X_test_clas)[:, 1]  # Probabilidad para la clase 1

# 5. Ajustar el umbral de decisión
umbral = 0.5  # Umbral ajustado
y_pred_tuned = (y_pred_proba >= umbral).astype(int)

# 6. Evaluar el modelo
print("Reporte de clasificación (umbral ajustado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (umbral ajustado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo (umbral ajustado):")
print(f"{roc_auc_score(y_test_clas, y_pred_proba):.4f}")

print("\nExactitud del modelo (umbral ajustado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")'''

c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:18:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Reporte de clasificación (umbral ajustado):
              precision    recall  f1-score   support

         0.0       0.88      0.77      0.82   1644104
         1.0       0.37      0.58      0.45    387781

    accuracy                           0.73   2031885
   macro avg       0.63      0.67      0.64   2031885
weighted avg       0.79      0.73      0.75   2031885


Matriz de confusión (umbral ajustado):
[[1260102  384002]
 [ 163785  223996]]

ROC-AUC del modelo (umbral ajustado):
0.7384

Exactitud del modelo (umbral ajustado):
73.04%


# EL MISMO QUE EL ANTERIOR PERO CON 2020, 2021, 2022 Y USANDO SOLO LOS HIPER PARAMETROS. SIN EL GRID SEARCH

In [ ]:
'''from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

# 2. Configurar el clasificador XGBoost
xgb_clasificador = XGBClassifier(
    objective='binary:logistic',  # Clasificación binaria
    eval_metric='logloss',       # Métrica para evaluar la pérdida
    use_label_encoder=False,     # Deshabilitar codificación automática
    random_state=42,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=200,
    scale_pos_weight=3.5765676028503317
)

# 3. Entrenar el modelo con los datos balanceados
xgb_clasificador.fit(X_train_clas, y_train_clas)

# 4. Obtener las probabilidades de predicción en el conjunto de prueba
y_pred_proba = xgb_clasificador.predict_proba(X_test_clas)[:, 1]  # Probabilidad para la clase 1

# 5. Ajustar el umbral de decisión
umbral = 0.5  # Umbral ajustado
y_pred_tuned = (y_pred_proba >= umbral).astype(int)

# 6. Evaluar el modelo
print("Reporte de clasificación (umbral ajustado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (umbral ajustado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo (umbral ajustado):")
print(f"{roc_auc_score(y_test_clas, y_pred_proba):.4f}")

print("\nExactitud del modelo (umbral ajustado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")'''

# GRIDSEARCH CON XGBOOST CON SMOTE

In [ ]:
'''#1. Aplicar oversampling con SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# Parámetros para buscar el mejor ajuste
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'scale_pos_weight': [1, len(y_train_clas[y_train_clas == 0]) / len(y_train_clas[y_train_clas == 1])]
}

# Crear el modelo base
xgb_clasificador_tuned = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(
    estimator=xgb_clasificador_tuned,
    param_grid=param_grid,
    scoring='roc_auc',  # Usamos ROC-AUC como métrica
    cv=3,  # Validación cruzada de 3 pliegues
    verbose=1,
    n_jobs=-1
)

# Entrenar la búsqueda
grid_search.fit(X_train_clas_bal, y_train_clas_bal)

# Mostrar los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

# Usar el mejor modelo para predecir
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_clas)
y_pred_proba_tuned = best_model.predict_proba(X_test_clas)[:, 1]

# Evaluación
print("\nReporte de clasificación (modelo optimizado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (modelo optimizado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo optimizado:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_tuned):.4f}")
'''

Fitting 3 folds for each of 24 candidates, totalling 72 fits


c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:59:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'scale_pos_weight': 1}

Reporte de clasificación (modelo optimizado):
              precision    recall  f1-score   support

         0.0       0.84      0.87      0.85    618487
         1.0       0.45      0.39      0.42    172927

    accuracy                           0.76    791414
   macro avg       0.64      0.63      0.64    791414
weighted avg       0.75      0.76      0.76    791414


Matriz de confusión (modelo optimizado):
[[536822  81665]
 [105441  67486]]

ROC-AUC del modelo optimizado:
0.7100


# XGBOOST SIN SMOTE (Y LOS MEJORES HIPERPARAMETROS)

In [ ]:
'''from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

# 2. Configurar el clasificador XGBoost
xgb_clasificador = XGBClassifier(
    objective='binary:logistic',  # Clasificación binaria
    eval_metric='logloss',       # Métrica para evaluar la pérdida
    use_label_encoder=False,     # Deshabilitar codificación automática
    random_state=42,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=200,
    scale_pos_weight=1
)

# 3. Entrenar el modelo con los datos balanceados
xgb_clasificador.fit(X_train_clas, y_train_clas)

# 4. Obtener las probabilidades de predicción en el conjunto de prueba
y_pred_proba = xgb_clasificador.predict_proba(X_test_clas)[:, 1]  # Probabilidad para la clase 1

# 5. Ajustar el umbral de decisión
umbral = 0.5  # Umbral ajustado
y_pred_tuned = (y_pred_proba >= umbral).astype(int)

# 6. Evaluar el modelo
print("Reporte de clasificación (umbral ajustado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (umbral ajustado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo (umbral ajustado):")
print(f"{roc_auc_score(y_test_clas, y_pred_proba):.4f}")

print("\nExactitud del modelo (umbral ajustado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")'''

c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:45:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Reporte de clasificación (umbral ajustado):
              precision    recall  f1-score   support

         0.0       0.81      0.98      0.88    618487
         1.0       0.65      0.17      0.26    172927

    accuracy                           0.80    791414
   macro avg       0.73      0.57      0.57    791414
weighted avg       0.77      0.80      0.75    791414


Matriz de confusión (umbral ajustado):
[[603075  15412]
 [144270  28657]]

ROC-AUC del modelo (umbral ajustado):
0.7350

Exactitud del modelo (umbral ajustado):
79.82%


# XGBOOST CON SMOTE (Y LOS MEJORES HIPERPARAMETROS)

In [101]:
'''from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

# 1. Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# 2. Configurar el clasificador XGBoost
xgb_clasificador = XGBClassifier(
    objective='binary:logistic',  # Clasificación binaria
    eval_metric='logloss',       # Métrica para evaluar la pérdida
    use_label_encoder=False,     # Deshabilitar codificación automática
    random_state=42,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=200,
    scale_pos_weight=1
)

# 3. Entrenar el modelo con los datos balanceados
xgb_clasificador.fit(X_train_clas_bal, y_train_clas_bal)

# 4. Obtener las probabilidades de predicción en el conjunto de prueba
y_pred_proba = xgb_clasificador.predict_proba(X_test_clas)[:, 1]  # Probabilidad para la clase 1

# 5. Ajustar el umbral de decisión
umbral = 0.5  # Umbral ajustado
y_pred_tuned = (y_pred_proba >= umbral).astype(int)

# 6. Evaluar el modelo
print("Reporte de clasificación (umbral ajustado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (umbral ajustado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo (umbral ajustado):")
print(f"{roc_auc_score(y_test_clas, y_pred_proba):.4f}")

print("\nExactitud del modelo (umbral ajustado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")'''

c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:53:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Reporte de clasificación (umbral ajustado):
              precision    recall  f1-score   support

         0.0       0.84      0.87      0.85    618487
         1.0       0.45      0.39      0.42    172927

    accuracy                           0.76    791414
   macro avg       0.64      0.63      0.64    791414
weighted avg       0.75      0.76      0.76    791414


Matriz de confusión (umbral ajustado):
[[536822  81665]
 [105441  67486]]

ROC-AUC del modelo (umbral ajustado):
0.7100

Exactitud del modelo (umbral ajustado):
76.36%


# XGBOOST CON SMOTE (Y LOS MEJORES HIPERPARAMETROS + 100 MAS PESO para 1)

In [107]:
'''from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

# 1. Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# 2. Configurar el clasificador XGBoost
xgb_clasificador = XGBClassifier(
    objective='binary:logistic',  # Clasificación binaria
    eval_metric='logloss',       # Métrica para evaluar la pérdida
    use_label_encoder=False,     # Deshabilitar codificación automática
    random_state=42,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=200,
    scale_pos_weight=100
)

# 3. Entrenar el modelo con los datos balanceados
xgb_clasificador.fit(X_train_clas_bal, y_train_clas_bal)

# 4. Obtener las probabilidades de predicción en el conjunto de prueba
y_pred_proba = xgb_clasificador.predict_proba(X_test_clas)[:, 1]  # Probabilidad para la clase 1

# 5. Ajustar el umbral de decisión
umbral = 0.5  # Umbral ajustado
y_pred_tuned = (y_pred_proba >= umbral).astype(int)

# 6. Evaluar el modelo
print("Reporte de clasificación (umbral ajustado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (umbral ajustado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo (umbral ajustado):")
print(f"{roc_auc_score(y_test_clas, y_pred_proba):.4f}")

print("\nExactitud del modelo (umbral ajustado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")'''

c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:53:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Reporte de clasificación (umbral ajustado):
              precision    recall  f1-score   support

         0.0       0.92      0.00      0.00    618487
         1.0       0.22      1.00      0.36    172927

    accuracy                           0.22    791414
   macro avg       0.57      0.50      0.18    791414
weighted avg       0.76      0.22      0.08    791414


Matriz de confusión (umbral ajustado):
[[   144 618343]
 [    13 172914]]

ROC-AUC del modelo (umbral ajustado):
0.7085

Exactitud del modelo (umbral ajustado):
21.87%


# XGBOOST CON SMOTE (Y LOS MEJORES HIPERPARAMETROS + 50 MAS PESO para 1)

In [58]:
'''from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

# 1. Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# 2. Configurar el clasificador XGBoost
xgb_clasificador = XGBClassifier(
    objective='binary:logistic',  # Clasificación binaria
    eval_metric='logloss',       # Métrica para evaluar la pérdida
    use_label_encoder=False,     # Deshabilitar codificación automática
    random_state=42,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=200,
    scale_pos_weight=50
)

# 3. Entrenar el modelo con los datos balanceados
xgb_clasificador.fit(X_train_clas_bal, y_train_clas_bal)

# 4. Obtener las probabilidades de predicción en el conjunto de prueba
y_pred_proba = xgb_clasificador.predict_proba(X_test_clas)[:, 1]  # Probabilidad para la clase 1

# 5. Ajustar el umbral de decisión
umbral = 0.5  # Umbral ajustado
y_pred_tuned = (y_pred_proba >= umbral).astype(int)

# 6. Evaluar el modelo
print("Reporte de clasificación (umbral ajustado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (umbral ajustado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo (umbral ajustado):")
print(f"{roc_auc_score(y_test_clas, y_pred_proba):.4f}")

print("\nExactitud del modelo (umbral ajustado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")'''

c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:07:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Reporte de clasificación (umbral ajustado):
              precision    recall  f1-score   support

         0.0       0.93      0.00      0.00    618487
         1.0       0.22      1.00      0.36    172927

    accuracy                           0.22    791414
   macro avg       0.57      0.50      0.18    791414
weighted avg       0.77      0.22      0.08    791414


Matriz de confusión (umbral ajustado):
[[   278 618209]
 [    22 172905]]

ROC-AUC del modelo (umbral ajustado):
0.7087

Exactitud del modelo (umbral ajustado):
21.88%


# XGBOOST CON SMOTE (Y LOS MEJORES HIPERPARAMETROS + PESO BALANCEADO)

In [ ]:
'''from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

# 1. Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

from collections import Counter

# Calcular el número de muestras en cada clase
class_counts = Counter(y_train_clas)
majority_class = max(class_counts, key=class_counts.get)
minority_class = min(class_counts, key=class_counts.get)

# Calcular el ratio entre clases
ratio_of_classes = class_counts[majority_class] / class_counts[minority_class]

# 2. Configurar el clasificador XGBoost
xgb_clasificador = XGBClassifier(
    objective='binary:logistic',  # Clasificación binaria
    eval_metric='logloss',       # Métrica para evaluar la pérdida
    use_label_encoder=False,     # Deshabilitar codificación automática
    random_state=42,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=200,
    scale_pos_weight=ratio_of_classes
)

# 3. Entrenar el modelo con los datos balanceados
xgb_clasificador.fit(X_train_clas_bal, y_train_clas_bal)

# 4. Obtener las probabilidades de predicción en el conjunto de prueba
y_pred_proba = xgb_clasificador.predict_proba(X_test_clas)[:, 1]  # Probabilidad para la clase 1

# 5. Ajustar el umbral de decisión
umbral = 0.5  # Umbral ajustado
y_pred_tuned = (y_pred_proba >= umbral).astype(int)

# 6. Evaluar el modelo
print("Reporte de clasificación (umbral ajustado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (umbral ajustado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo (umbral ajustado):")
print(f"{roc_auc_score(y_test_clas, y_pred_proba):.4f}")

print("\nExactitud del modelo (umbral ajustado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")'''

c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:26:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Reporte de clasificación (umbral ajustado):
              precision    recall  f1-score   support

         0.0       0.90      0.37      0.52    618487
         1.0       0.28      0.86      0.42    172927

    accuracy                           0.47    791414
   macro avg       0.59      0.61      0.47    791414
weighted avg       0.77      0.47      0.50    791414


Matriz de confusión (umbral ajustado):
[[226528 391959]
 [ 23938 148989]]

ROC-AUC del modelo (umbral ajustado):
0.7127

Exactitud del modelo (umbral ajustado):
47.45%


# XGBOOST CON ADASYN (Y LOS MEJORES HIPERPARAMETROS)

In [ ]:
'''from imblearn.over_sampling import ADASYN
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

# 1. Aplicar ADASYN para balancear las clases
adasyn = ADASYN(random_state=42)
X_train_clas_bal, y_train_clas_bal = adasyn.fit_resample(X_train_clas, y_train_clas)

# 2. Configurar el clasificador XGBoost
xgb_clasificador = XGBClassifier(
    objective='binary:logistic',  # Clasificación binaria
    eval_metric='logloss',       # Métrica para evaluar la pérdida
    use_label_encoder=False,     # Deshabilitar codificación automática
    random_state=42,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=200,
    scale_pos_weight=1
)

# 3. Entrenar el modelo con los datos balanceados
xgb_clasificador.fit(X_train_clas_bal, y_train_clas_bal)

# 4. Obtener las probabilidades de predicción en el conjunto de prueba
y_pred_proba = xgb_clasificador.predict_proba(X_test_clas)[:, 1]  # Probabilidad para la clase 1

# 5. Ajustar el umbral de decisión
umbral = 0.5  # Umbral ajustado
y_pred_tuned = (y_pred_proba >= umbral).astype(int)

# 6. Evaluar el modelo
print("Reporte de clasificación (umbral ajustado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (umbral ajustado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo (umbral ajustado):")
print(f"{roc_auc_score(y_test_clas, y_pred_proba):.4f}")

print("\nExactitud del modelo (umbral ajustado):")
print(f"{accuracy_score(y_test_clas, y_pred_tuned) * 100:.2f}%")
'''

c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:39:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Reporte de clasificación (umbral ajustado):
              precision    recall  f1-score   support

         0.0       0.83      0.88      0.86    618487
         1.0       0.46      0.36      0.41    172927

    accuracy                           0.77    791414
   macro avg       0.65      0.62      0.63    791414
weighted avg       0.75      0.77      0.76    791414


Matriz de confusión (umbral ajustado):
[[546774  71713]
 [110729  62198]]

ROC-AUC del modelo (umbral ajustado):
0.7070

Exactitud del modelo (umbral ajustado):
76.95%


# GRIDSEARCH CON XGBOOST SIN SMOTE

In [103]:
'''# Parámetros para buscar el mejor ajuste
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'scale_pos_weight': [1, len(y_train_clas[y_train_clas == 0]) / len(y_train_clas[y_train_clas == 1])]
}

# Crear el modelo base
xgb_clasificador_tuned = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(
    estimator=xgb_clasificador_tuned,
    param_grid=param_grid,
    scoring='roc_auc',  # Usamos ROC-AUC como métrica
    cv=3,  # Validación cruzada de 3 pliegues
    verbose=1,
    n_jobs=-1
)

# Entrenar la búsqueda
grid_search.fit(X_train_clas, y_train_clas)

# Mostrar los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

# Usar el mejor modelo para predecir
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_clas)
y_pred_proba_tuned = best_model.predict_proba(X_test_clas)[:, 1]

# Evaluación
print("\nReporte de clasificación (modelo optimizado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (modelo optimizado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo optimizado:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_tuned):.4f}")'''


'# Parámetros para buscar el mejor ajuste\nparam_grid = {\n    \'max_depth\': [3, 5, 7],\n    \'learning_rate\': [0.01, 0.1],\n    \'n_estimators\': [100, 200],\n    \'scale_pos_weight\': [1, len(y_train_clas[y_train_clas == 0]) / len(y_train_clas[y_train_clas == 1])]\n}\n\n# Crear el modelo base\nxgb_clasificador_tuned = XGBClassifier(\n    objective=\'binary:logistic\',\n    eval_metric=\'logloss\',\n    use_label_encoder=False,\n    random_state=42\n)\n\n# Búsqueda de hiperparámetros con validación cruzada\ngrid_search = GridSearchCV(\n    estimator=xgb_clasificador_tuned,\n    param_grid=param_grid,\n    scoring=\'roc_auc\',  # Usamos ROC-AUC como métrica\n    cv=3,  # Validación cruzada de 3 pliegues\n    verbose=1,\n    n_jobs=-1\n)\n\n# Entrenar la búsqueda\ngrid_search.fit(X_train_clas, y_train_clas)\n\n# Mostrar los mejores parámetros\nprint("Mejores parámetros:", grid_search.best_params_)\n\n# Usar el mejor modelo para predecir\nbest_model = grid_search.best_estimator_\ny_pre

# RANDOMIZED CON XGBBOOST

In [104]:
'''from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import numpy as np

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# Definir parámetros para la búsqueda aleatoria
param_dist = {
    'max_depth': [3, 5, 7, 9],  # Mayor rango para mayor flexibilidad
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150, 200],
    'scale_pos_weight': np.linspace(1, 10, 5)  # Valores balanceados automáticamente
}

# Crear el modelo base
xgb_clasificador_tuned = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Búsqueda de hiperparámetros con RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_clasificador_tuned,
    param_distributions=param_dist,
    n_iter=10,  # Número de combinaciones a probar
    scoring='roc_auc',  # Usamos ROC-AUC como métrica
    cv=3,  # Validación cruzada de 3 pliegues
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Entrenar la búsqueda
random_search.fit(X_train_clas_bal, y_train_clas_bal)

# Mostrar los mejores parámetros
print("Mejores parámetros:", random_search.best_params_)

# Usar el mejor modelo para predecir
best_model = random_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_clas)
y_pred_proba_tuned = best_model.predict_proba(X_test_clas)[:, 1]

# Evaluación
print("\nReporte de clasificación (modelo optimizado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (modelo optimizado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo optimizado:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_tuned):.4f}")
'''

'from sklearn.model_selection import RandomizedSearchCV\nfrom xgboost import XGBClassifier\nfrom imblearn.over_sampling import SMOTE\nfrom sklearn.metrics import classification_report, confusion_matrix, roc_auc_score\nimport numpy as np\n\n# Aplicar SMOTE para balancear las clases\nsmote = SMOTE(random_state=42)\nX_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)\n\n# Definir parámetros para la búsqueda aleatoria\nparam_dist = {\n    \'max_depth\': [3, 5, 7, 9],  # Mayor rango para mayor flexibilidad\n    \'learning_rate\': [0.01, 0.05, 0.1, 0.2],\n    \'n_estimators\': [50, 100, 150, 200],\n    \'scale_pos_weight\': np.linspace(1, 10, 5)  # Valores balanceados automáticamente\n}\n\n# Crear el modelo base\nxgb_clasificador_tuned = XGBClassifier(\n    objective=\'binary:logistic\',\n    eval_metric=\'logloss\',\n    use_label_encoder=False,\n    random_state=42\n)\n\n# Búsqueda de hiperparámetros con RandomizedSearchCV\nrandom_search = RandomizedSearchCV

# NAIVE BAYES

In [60]:
'''from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1. Crear el modelo de Naive Bayes
gaussian_nb = GaussianNB()

# 2. Entrenar el modelo con los datos de entrenamiento
gaussian_nb.fit(X_train_clas, y_train_clas)

# 3. Predecir en el conjunto de prueba
y_pred_clas = gaussian_nb.predict(X_test_clas)

# 4. Evaluar el modelo
print("Reporte de clasificación:")
print(classification_report(y_test_clas, y_pred_clas))

print("\nMatriz de confusión:")
print(confusion_matrix(y_test_clas, y_pred_clas))

print("\nExactitud del modelo:")
print(f"{accuracy_score(y_test_clas, y_pred_clas) * 100:.2f}%")
'''

Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.78      0.99      0.88    618487
         1.0       0.46      0.02      0.03    172927

    accuracy                           0.78    791414
   macro avg       0.62      0.51      0.45    791414
weighted avg       0.71      0.78      0.69    791414


Matriz de confusión:
[[614998   3489]
 [169944   2983]]

Exactitud del modelo:
78.09%


# LOGISTIC REGRESSION

In [64]:
'''from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# 1. Crear el modelo de Regresión Logística
logistic_reg = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')

# 2. Entrenar el modelo con los datos de entrenamiento
logistic_reg.fit(X_train_clas_bal, y_train_clas_bal)

# 3. Predecir en el conjunto de prueba
y_pred_clas = logistic_reg.predict(X_test_clas)

# 4. Evaluar el modelo
print("Reporte de clasificación:")
print(classification_report(y_test_clas, y_pred_clas))

print("\nMatriz de confusión:")
print(confusion_matrix(y_test_clas, y_pred_clas))

print("\nExactitud del modelo:")
print(f"{accuracy_score(y_test_clas, y_pred_clas) * 100:.2f}%")
'''

c:\Users\manue\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.83      0.64      0.72    618487
         1.0       0.29      0.51      0.37    172927

    accuracy                           0.62    791414
   macro avg       0.56      0.58      0.55    791414
weighted avg       0.71      0.62      0.65    791414


Matriz de confusión:
[[398416 220071]
 [ 83871  89056]]

Exactitud del modelo:
61.60%


# LINEAR REGRESSION

# GRID CON LIGHTGBM

In [105]:
'''from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

# 1. Aplicar oversampling con SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# Parámetros para buscar el mejor ajuste
param_grid = {
    'max_depth': [3, 5, 7],              # Profundidad máxima de los árboles
    'learning_rate': [0.01, 0.1],        # Tasa de aprendizaje
    'n_estimators': [100, 200]           # Número de árboles
}

# Crear el modelo base
lgbm_clasificador_tuned = LGBMClassifier(
    objective='binary',                # Clasificación binaria"
    random_state=42,                   # Semilla para reproducibilidad
    class_weight='balanced'                  # Desactivamos el balanceo automático para usar scale_pos_weight
)

# Búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(
    estimator=lgbm_clasificador_tuned,
    param_grid=param_grid,
    scoring='roc_auc',                 # Usamos ROC-AUC como métrica
    cv=3,                              # Validación cruzada de 3 pliegues
    verbose=1,                         # Mostrar progreso
    n_jobs=-1                          # Usar todos los núcleos disponibles
)

# Entrenar la búsqueda
grid_search.fit(X_train_clas_bal, y_train_clas_bal)

# Mostrar los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

# Usar el mejor modelo para predecir
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_clas)
y_pred_proba_tuned = best_model.predict_proba(X_test_clas)[:, 1]

# Evaluación
print("\nReporte de clasificación (modelo optimizado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (modelo optimizado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo optimizado:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_tuned):.4f}")
'''

'from lightgbm import LGBMClassifier\nfrom sklearn.metrics import classification_report, confusion_matrix, roc_auc_score\nfrom sklearn.model_selection import GridSearchCV\nfrom imblearn.over_sampling import SMOTE\n\n# 1. Aplicar oversampling con SMOTE al conjunto de entrenamiento\nsmote = SMOTE(random_state=42)\nX_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)\n\n# Parámetros para buscar el mejor ajuste\nparam_grid = {\n    \'max_depth\': [3, 5, 7],              # Profundidad máxima de los árboles\n    \'learning_rate\': [0.01, 0.1],        # Tasa de aprendizaje\n    \'n_estimators\': [100, 200]           # Número de árboles\n}\n\n# Crear el modelo base\nlgbm_clasificador_tuned = LGBMClassifier(\n    objective=\'binary\',                # Clasificación binaria"\n    random_state=42,                   # Semilla para reproducibilidad\n    class_weight=\'balanced\'                  # Desactivamos el balanceo automático para usar scale_pos_weight\n)\n\n# 

# RANDOMIZED CON LIGHTGBM

In [106]:
'''from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
import numpy as np

# 1. Aplicar oversampling con SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# Parámetros para buscar el mejor ajuste
param_dist = {
    'max_depth': [3, 5, 7, 9, None],              # Profundidad máxima de los árboles
    'learning_rate': np.linspace(0.01, 0.2, 10), # Tasa de aprendizaje en un rango
    'n_estimators': [50, 100, 150, 200],         # Número de árboles
    'num_leaves': [20, 31, 40, 50],              # Número de hojas
    'min_child_samples': [10, 20, 30, 50],       # Mínimo de muestras en un nodo
    'subsample': np.linspace(0.6, 1.0, 5),       # Submuestreo de datos
    'colsample_bytree': np.linspace(0.6, 1.0, 5) # Submuestreo de características
}

# Crear el modelo base
lgbm_clasificador_tuned = LGBMClassifier(
    objective='binary',                # Clasificación binaria
    random_state=42,                   # Semilla para reproducibilidad
    class_weight='balanced'            # Ajuste automático para clases desbalanceadas
)

# Búsqueda de hiperparámetros con RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=lgbm_clasificador_tuned,
    param_distributions=param_dist,
    n_iter=10,                         # Número de combinaciones aleatorias a probar
    scoring='roc_auc',                 # Usamos ROC-AUC como métrica
    cv=3,                              # Validación cruzada de 3 pliegues
    verbose=1,                         # Mostrar progreso
    n_jobs=-1,                         # Usar todos los núcleos disponibles
    random_state=42                    # Semilla para reproducibilidad
)

# Entrenar la búsqueda
random_search.fit(X_train_clas_bal, y_train_clas_bal)

# Mostrar los mejores parámetros
print("Mejores parámetros:", random_search.best_params_)

# Usar el mejor modelo para predecir
best_model = random_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_clas)
y_pred_proba_tuned = best_model.predict_proba(X_test_clas)[:, 1]

# Evaluación
print("\nReporte de clasificación (modelo optimizado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (modelo optimizado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo optimizado:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_tuned):.4f}")
'''

'from lightgbm import LGBMClassifier\nfrom sklearn.metrics import classification_report, confusion_matrix, roc_auc_score\nfrom sklearn.model_selection import RandomizedSearchCV\nfrom imblearn.over_sampling import SMOTE\nimport numpy as np\n\n# 1. Aplicar oversampling con SMOTE al conjunto de entrenamiento\nsmote = SMOTE(random_state=42)\nX_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)\n\n# Parámetros para buscar el mejor ajuste\nparam_dist = {\n    \'max_depth\': [3, 5, 7, 9, None],              # Profundidad máxima de los árboles\n    \'learning_rate\': np.linspace(0.01, 0.2, 10), # Tasa de aprendizaje en un rango\n    \'n_estimators\': [50, 100, 150, 200],         # Número de árboles\n    \'num_leaves\': [20, 31, 40, 50],              # Número de hojas\n    \'min_child_samples\': [10, 20, 30, 50],       # Mínimo de muestras en un nodo\n    \'subsample\': np.linspace(0.6, 1.0, 5),       # Submuestreo de datos\n    \'colsample_bytree\': np.linspace(0